# Лабораторная работа №1. Алгоритм k ближайших соседей (KNN)

## Задачи
1. Построить базовые модели KNN для:
   - классификации съедобности грибов 
   - регрессии популярности треков
2. Оценить качество моделей по выбранным метрикам
3. Сформулировать и проверить гипотезы по улучшению бейзлайна (препроцессинг, подбор гиперпараметров и т.д.)
4. Построить улучшенный бейзлайн и сравнить его с исходным
5. Самостоятельно имплементировать KNN (классификация и регрессия), обучить и сравнить с реализацией из `sklearn`


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from pathlib import Path

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_absolute_error, mean_squared_error, r2_score,
    classification_report
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [4]:

mushroom_path = "data/mushroom-data.csv"
spotify_path = "data/modified-spotify-dataset.csv"

mushroom_df = pd.read_csv(mushroom_path)
spotify_df = pd.read_csv(spotify_path)

mushroom_df.head(), spotify_df.head()


(  Mushroom_quality cap_shape cap_surface cap_color bruises odor  \
 0                p         x           s         n       t    p   
 1                e         x           s         y       t    a   
 2                e         b           s         w       t    l   
 3                p         x           y         w       t    p   
 4                e         x           s         g       f    n   
 
   gill_attachment gill_spacing gill_size gill_color  ...  \
 0               f            c         n          k  ...   
 1               f            c         b          k  ...   
 2               f            c         b          n  ...   
 3               f            c         n          n  ...   
 4               f            w         b          k  ...   
 
   stalk_surface_below_ring stalk_color_above_ring stalk_color_below_ring  \
 0                        s                      w                      w   
 1                        s                      w                  

In [5]:
target_col_mush = "Mushroom_quality"

X_mush = mushroom_df.drop(columns=[target_col_mush])
y_mush = mushroom_df[target_col_mush]

X_mush_train, X_mush_test, y_mush_train, y_mush_test = train_test_split(
    X_mush, y_mush, test_size=0.2, random_state=RANDOM_STATE, stratify=y_mush
)

X_mush_train.shape, X_mush_test.shape


((6499, 22), (1625, 22))

In [6]:
target_col_spot = "popularity"

cols_to_drop = [
    "track_id", "track_name", "artist_name", "album_name","popularity_level"  
]
cols_to_drop = [c for c in cols_to_drop if c in spotify_df.columns]

features_spot_df = spotify_df.drop(columns=[target_col_spot] + cols_to_drop)
y_spot = spotify_df[target_col_spot]

numeric_features_spot = features_spot_df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features_spot = features_spot_df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

X_spot_train, X_spot_test, y_spot_train, y_spot_test = train_test_split(
    features_spot_df, y_spot, test_size=0.2, random_state=RANDOM_STATE
)

len(numeric_features_spot), len(categorical_features_spot)


(156, 1)

In [7]:
categorical_features_mush = X_mush_train.columns.tolist()

mush_preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features_mush)
    ]
)

knn_clf_baseline = Pipeline(
    steps=[
        ("preprocess", mush_preprocess),
        ("model", KNeighborsClassifier(n_neighbors=5))
    ]
)

knn_clf_baseline.fit(X_mush_train, y_mush_train)

y_mush_pred = knn_clf_baseline.predict(X_mush_test)

acc = accuracy_score(y_mush_test, y_mush_pred)
prec = precision_score(y_mush_test, y_mush_pred, pos_label=y_mush_train.unique()[0], average='binary')
rec = recall_score(y_mush_test, y_mush_pred, pos_label=y_mush_train.unique()[0], average='binary')
f1 = f1_score(y_mush_test, y_mush_pred, pos_label=y_mush_train.unique()[0], average='binary')

print("=== Baseline KNN (Mushrooms) ===")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification report:")
print(classification_report(y_mush_test, y_mush_pred))


=== Baseline KNN (Mushrooms) ===
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-score: 1.0000

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



In [10]:
# Препроцессинг для Spotify
from sklearn.metrics import root_mean_squared_error


spot_preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features_spot),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features_spot),
    ]
)

knn_reg_baseline = Pipeline(
    steps=[
        ("preprocess", spot_preprocess),
        ("model", KNeighborsRegressor(n_neighbors=5))
    ]
)

knn_reg_baseline.fit(X_spot_train, y_spot_train)

y_spot_pred = knn_reg_baseline.predict(X_spot_test)

mae = mean_absolute_error(y_spot_test, y_spot_pred)
rmse = root_mean_squared_error(y_spot_test, y_spot_pred)
r2 = r2_score(y_spot_test, y_spot_pred)

print("=== Baseline KNN (Spotify) ===")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.4f}")

=== Baseline KNN (Spotify) ===
MAE: 0.3571
RMSE: 0.5603
R2: 0.6852


## Улучшение бейзлайна: формулировка гипотез

Гипотезы, которые проверим:

1. **Масштабирование числовых признаков улучшит качество KNN**, так как алгоритм чувствителен к масштабу
2. **Подбор числа соседей `n_neighbors` и метрики расстояния** через кросс-валидацию даст выигрыш по качеству
3. Для Spotify:
   - возможно, стоит ограничить набор признаков (удалить слабосвязанные фичи),
   - попробовать разные вес


In [11]:
param_grid_mush = {
    "model__n_neighbors": [3, 5, 7, 9, 11],
    "model__weights": ["uniform", "distance"],
    "model__p": [1, 2]  # манхэттен / евклид
}

knn_clf_pipeline = Pipeline(
    steps=[
        ("preprocess", mush_preprocess),
        ("model", KNeighborsClassifier())
    ]
)

grid_mush = GridSearchCV(
    knn_clf_pipeline,
    param_grid=param_grid_mush,
    cv=3,
    scoring="f1_macro",
    n_jobs=-1
)

grid_mush.fit(X_mush_train, y_mush_train)

print("Лучшие параметры (грибы):", grid_mush.best_params_)

best_clf = grid_mush.best_estimator_
y_mush_pred_best = best_clf.predict(X_mush_test)

acc_b = accuracy_score(y_mush_test, y_mush_pred_best)
prec_b = precision_score(y_mush_test, y_mush_pred_best, average='macro')
rec_b = recall_score(y_mush_test, y_mush_pred_best, average='macro')
f1_b = f1_score(y_mush_test, y_mush_pred_best, average='macro')

print("=== Improved KNN (Mushrooms) ===")
print(f"Accuracy: {acc_b:.4f}")
print(f"Precision (macro): {prec_b:.4f}")
print(f"Recall (macro): {rec_b:.4f}")
print(f"F1-score (macro): {f1_b:.4f}")


c:\ProgramData\.anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [       nan 0.99922978 0.99922978 0.99922978        nan 0.99922978
 0.99922978 0.99922978        nan 0.99922978 0.99922978 0.99922978
        nan 0.99922978 0.99922978 0.99922978        nan 0.99922978
 0.99922978 0.99922978]
  warnings.warn(


Лучшие параметры (грибы): {'model__n_neighbors': 3, 'model__p': 1, 'model__weights': 'distance'}
=== Improved KNN (Mushrooms) ===
Accuracy: 1.0000
Precision (macro): 1.0000
Recall (macro): 1.0000
F1-score (macro): 1.0000


In [26]:
# Улучшенный KNN для Spotify: масштабирование + подбор гиперпараметров

spot_preprocess_scaled = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features_spot),
        # ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features_spot),
    ]
)

knn_reg_pipeline = Pipeline(
    steps=[
        ("preprocess", spot_preprocess_scaled),
        ("model", KNeighborsRegressor())
    ]
)

param_grid_spot = {
    "model__n_neighbors": [5, 10],
    "model__weights": ["distance"],
    "model__p": [2],  # только евклид
}

grid_spot = GridSearchCV(
    knn_reg_pipeline,
    param_grid=param_grid_spot,
    cv=2,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

grid_spot.fit(X_spot_train, y_spot_train)

print("Лучшие параметры (Spotify):", grid_spot.best_params_)

best_reg = grid_spot.best_estimator_
y_spot_pred_best = best_reg.predict(X_spot_test)

mae_b = mean_absolute_error(y_spot_test, y_spot_pred_best)
rmse_b = root_mean_squared_error(y_spot_test, y_spot_pred_best)
r2_b = r2_score(y_spot_test, y_spot_pred_best)

print("=== Improved KNN (Spotify) ===")
print(f"MAE: {mae_b:.4f}")
print(f"RMSE: {rmse_b:.4f}")
print(f"R2: {r2_b:.4f}")


Лучшие параметры (Spotify): {'model__n_neighbors': 10, 'model__p': 2, 'model__weights': 'distance'}
=== Improved KNN (Spotify) ===
MAE: 0.3808
RMSE: 0.5858
R2: 0.6558


In [27]:
results_mush = pd.DataFrame({
    "model": ["baseline", "improved"],
    "accuracy": [acc, acc_b],
    "f1_macro": [
        f1_score(y_mush_test, y_mush_pred, average="macro"),
        f1_b
    ]
})

results_spot = pd.DataFrame({
    "model": ["baseline", "improved"],
    "MAE": [mae, mae_b],
    "RMSE": [rmse, rmse_b],
    "R2": [r2, r2_b]
})

print("=== Mushrooms ===")
print(results_mush)
print("\n=== Spotify ===")
print(results_spot)


=== Mushrooms ===
      model  accuracy  f1_macro
0  baseline       1.0       1.0
1  improved       1.0       1.0

=== Spotify ===
      model       MAE      RMSE        R2
0  baseline  0.357071  0.560296  0.685171
1  improved  0.380839  0.585831  0.655822


In [33]:
from collections import Counter
from scipy import sparse

class MyKNN:
    def __init__(self, n_neighbors=5, mode="classification"):
        self.n_neighbors = n_neighbors
        self.mode = mode
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        # X может быть sparse → переведём в плотный формат
        if sparse.issparse(X):
            X = X.toarray()
        self.X_train = np.asarray(X)
        self.y_train = np.asarray(y)
        return self

    def _predict_one(self, x):
        # x — одномерный вектор признаков
        distances = np.linalg.norm(self.X_train - x, axis=1)
        idx = np.argsort(distances)[: self.n_neighbors]
        neighbors_y = self.y_train[idx]
        if self.mode == "classification":
            counter = Counter(neighbors_y)
            return counter.most_common(1)[0][0]
        else:
            return neighbors_y.astype(float).mean()

    def predict(self, X):
        # корректная обработка 2D и sparse
        if sparse.issparse(X):
            X = X.toarray()
        X = np.asarray(X)
        if X.ndim == 1:
            X = X.reshape(1, -1)
        return np.array([self._predict_one(x) for x in X])


In [34]:
# Возьмём лучший препроцессор (из improved пайплайна, но можно и из baseline)
mush_preprocess_only = best_clf.named_steps["preprocess"]

X_mush_train_proc = mush_preprocess_only.fit_transform(X_mush_train)
X_mush_test_proc = mush_preprocess_only.transform(X_mush_test)

my_knn_clf = MyKNN(n_neighbors=grid_mush.best_params_["model__n_neighbors"], mode="classification")
my_knn_clf.fit(X_mush_train_proc, y_mush_train.values)

y_mush_pred_my = my_knn_clf.predict(X_mush_test_proc)

acc_my = accuracy_score(y_mush_test, y_mush_pred_my)
f1_my = f1_score(y_mush_test, y_mush_pred_my, average="macro")

print("=== MyKNN (Mushrooms) ===")
print(f"Accuracy: {acc_my:.4f}")
print(f"F1-macro: {f1_my:.4f}")


=== MyKNN (Mushrooms) ===
Accuracy: 1.0000
F1-macro: 1.0000


In [38]:
from sklearn.model_selection import train_test_split

spot_preprocess_only = best_reg.named_steps["preprocess"]

X_spot_small, _, y_spot_small, _ = train_test_split(
    X_spot_train, y_spot_train, train_size=5000, random_state=42
)
X_spot_small_test, _, y_spot_small_test, _ = train_test_split(
    X_spot_test, y_spot_test, train_size=2000, random_state=42
)

X_spot_train_proc = spot_preprocess_only.fit_transform(X_spot_small)
X_spot_test_proc = spot_preprocess_only.transform(X_spot_small_test)

my_knn_reg = MyKNN(
    n_neighbors=grid_spot.best_params_["model__n_neighbors"],
    mode="regression"
)
my_knn_reg.fit(X_spot_train_proc, y_spot_small.values)

y_spot_pred_my = my_knn_reg.predict(X_spot_test_proc)

mae_my = mean_absolute_error(y_spot_small_test, y_spot_pred_my)
rmse_my = root_mean_squared_error(y_spot_small_test, y_spot_pred_my)
r2_my = r2_score(y_spot_small_test, y_spot_pred_my)

print("=== MyKNN (Spotify, subsample) ===")
print(f"MAE: {mae_my:.4f}")
print(f"RMSE: {rmse_my:.4f}")
print(f"R2: {r2_my:.4f}")


=== MyKNN (Spotify, subsample) ===
MAE: 0.5397
RMSE: 0.7663
R2: 0.4386


In [39]:
compare_mush = pd.DataFrame({
    "model": ["sklearn_baseline", "sklearn_improved", "my_knn"],
    "accuracy": [
        acc,
        acc_b,
        acc_my
    ],
    "f1_macro": [
        f1_score(y_mush_test, y_mush_pred, average="macro"),
        f1_b,
        f1_my
    ]
})

compare_spot = pd.DataFrame({
    "model": ["sklearn_baseline", "sklearn_improved", "my_knn"],
    "MAE": [mae, mae_b, mae_my],
    "RMSE": [rmse, rmse_b, rmse_my],
    "R2": [r2, r2_b, r2_my]
})

print("=== Mushrooms: sklearn vs MyKNN ===")
print(compare_mush)

print("\n=== Spotify: sklearn vs MyKNN ===")
print(compare_spot)


=== Mushrooms: sklearn vs MyKNN ===
              model  accuracy  f1_macro
0  sklearn_baseline       1.0       1.0
1  sklearn_improved       1.0       1.0
2            my_knn       1.0       1.0

=== Spotify: sklearn vs MyKNN ===
              model       MAE      RMSE        R2
0  sklearn_baseline  0.357071  0.560296  0.685171
1  sklearn_improved  0.380839  0.585831  0.655822
2            my_knn  0.539669  0.766279  0.438612


## Выводы по лабораторной работе №1

### 1. Результаты базового и улучшенного бейзлайна

**Задача классификации (грибы):**
- Базовый и улучшенный KNN из `sklearn` показали идеальное качество:  
  **Accuracy = 1.0000, F1-macro = 1.0000**
- Объяснение: категориальные признаки после One-Hot Encoding создают разреженное пространство, где KNN идеально разделяет классы
- Подбор гиперпараметров не дал улучшения — базовая конфигурация оптимальна

**Задача регрессии (Spotify):**
| Модель | MAE   | RMSE  | R²     |
|--------|-------|-------|--------|
| Baseline | **0.3571** | **0.5603** | **0.6852** |
| Improved | 0.3808 | 0.5858 | 0.6558 |

- Базовый KNN показал хорошее качество, улучшенный бейзлайн ухудшился
- Причина: переобучение на полном датасете с большим числом признаков 

### 2. Сравнение собственной реализации MyKNN с sklearn

**Классификация (грибы):**
sklearn_baseline: Accuracy=1.0000, F1-macro=1.0000
sklearn_improved: Accuracy=1.0000, F1-macro=1.0000
my_knn: Accuracy=1.0000, F1-macro=1.0000 

**MyKNN полностью воспроизвела качество sklearn** — реализация корректна

**Регрессия (Spotify):**
| Модель | MAE   | RMSE  | R²     |
|--------|-------|-------|--------|
| sklearn_baseline | 0.3571 | 0.5603 | **0.6852** |
| sklearn_improved | 0.3808 | 0.5858 | 0.6558 |
| **my_knn** | **0.5397** | **0.7663** | **0.4386** |

**MyKNN показала ухудшение** из-за:
- Работы на подвыборке (квадратичная сложность O(n²))
- Отсутствия оптимизаций (KD-tree, векторизация)

### 3. Общие выводы

 **KNN отлично работает** на категориальных данных (грибы) с чётким разделением классов  
 **Для регрессии** KNN даёт приемлемое качество (R²≈0.65-0.69), но чувствителен к масштабу и числу признаков  
 **Собственная реализация подтвердила понимание алгоритма**, выявив практические ограничения скорости  
 **Метрики адекватно отразили поведение**: идеальные для классификации, умеренно хорошие для регрессии  

### 4. Практические рекомендации

**Для грибов**: KNN оптимален, следующие алгоритмы покажут ту же точность, но быстрее обучаются  
**Для Spotify**: ожидаем улучшений от ансамблей (Random Forest, Gradient Boosting) на нелинейных зависимостях  

**Вывод**: KNN — сильный baseline для небольших датасетов с категориальными признаками, но требует оптимизации для больших табличных данных.
